In [1]:
# Necessary and residual imports
import nltk
from nltk.corpus import udhr
import keras
from nltk import FreqDist
from gensim.models.word2vec import Word2Vec
import pandas as pd
import regex
import nltk.tokenize.casual
import tensorflow as tf

In [2]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import gc

import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load dataset: (from https://huggingface.co/datasets/versae/bibles)
# https://stackoverflow.com/questions/39263929/how-can-i-read-tar-gz-file-using-pandas-read-csv-with-gzip-compression-option
df = pd.read_csv('books_labels.tar.gz', compression='gzip', header=0, sep=',', quotechar='"', on_bad_lines='skip')

In [4]:
# Select Spanish texts
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
#spanish_df = df[df['language'] == 'SPA']['text']
#spanish_list = spanish_df.to_list() # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://ioflood.com/blog/dataframe-to-list-pandas/%23:~:text%3DIt%2520is%2520utilized%2520with%2520the,tolist()%2520.%26text%3DIn%2520the%2520example%2520above%252C%2520we,convert%2520it%2520into%2520a%2520list.&ved=2ahUKEwiP3deMnLSFAxVD4ckDHUo7BsEQFnoECA4QAw&usg=AOvVaw3GgfIVdsVo9Dxul02uata1

In [5]:
set(df['language'].to_list())

{'ALB',
 'ARA',
 'AZB',
 'BAQ',
 'BEL',
 'BLA',
 'BUL',
 'CEB',
 'CHA',
 'CHI',
 'CKB',
 'COP',
 'CZE',
 'DAN',
 'ENG',
 'ESP',
 'FIN',
 'FRE',
 'GAE',
 'GER',
 'GOT',
 'GRC',
 'HAT',
 'HEB',
 'HIN',
 'HUN',
 'ITA',
 'JAM',
 'KOR',
 'LAT',
 'MAR',
 'NDS',
 'NL_',
 'NOR',
 'PON',
 'POR',
 'RUM',
 'RUS',
 'SCR',
 'SHU',
 'SPA',
 'SWA',
 'SWE',
 'SYR',
 'TAM',
 'TGL',
 'THA',
 'TUR',
 'UND',
 'VIE',
 'WIU',
 'XKL',
 'YBY',
 'ZIA',
 nan}

In [6]:
set(df[df['language'] == 'ENG']['file_name_translation'].to_list())

{'SF_2004-04-25_ENG_MARCGAL_(THE EPISTLE TO THE GALATIANS(DETERING))',
 'SF_2009-01-20_ENG_ACV_(A CONSERVATIVE VERSION)',
 'SF_2009-01-20_ENG_ASV_(AMERICAN STANDARD VERSION)',
 'SF_2009-01-20_ENG_BBE_(BIBLE IN BASIC ENGLISH)',
 'SF_2009-01-20_ENG_BIBLE_AKJV_(AMERICAN KING JAMES VERSION)',
 'SF_2009-01-20_ENG_BIBLE_TYNDALE_(WILLAM TYNDALE BIBLE)',
 'SF_2009-01-20_ENG_DARBY_(DARBY BIBLE)',
 'SF_2009-01-20_ENG_DIAGLOT_(DIAGLOT NT - 1865)',
 'SF_2009-01-20_ENG_GB_(GENEVA BIBLE)',
 'SF_2009-01-20_ENG_JUBL2000_(ENGLISH JUBILEE 2000 BIBLE)',
 'SF_2009-01-20_ENG_KJ2000_(KING JAMES 2000)',
 'SF_2009-01-20_ENG_KJV_(KJV+)',
 'SF_2009-01-20_ENG_LEESER_(LEESER OLD TESTAMENT)',
 'SF_2009-01-20_ENG_LONT_(LIVING ORACLES NT)',
 "SF_2009-01-20_ENG_LXXE_(BRENTON'S ENGLISH SEPTUAGINT)",
 'SF_2009-01-20_ENG_MONT_(MONTGOMERY NEW TESTAMENT)',
 'SF_2009-01-20_ENG_NOYES_(NOYES TRANSLATION)',
 'SF_2009-01-20_ENG_NSB_(NEW SIMPLIFIED BIBLE)',
 'SF_2009-01-20_ENG_RNT_(THE RIVERSIDE NEW TESTAMENT)',
 'SF_2009-01-20

In [7]:
# Select Spanish texts
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
spanish_df = df[df['language'] == 'SPA']
portuguese_df = df[df['language'] == 'POR']
english_df = df[df['language'] == 'ENG']
german_df = df[df['language'] == 'GER']

spanish_df = spanish_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'Nueva Biblia de las Américas (NBLA)'] # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.reddit.com/r/Reformed/comments/xoye82/most_accurate_spanish_bible_translations_spanish/&ved=2ahUKEwj7qbSKtbuFAxV4IEQIHbm6B_UQFnoECCwQAQ&usg=AOvVaw2YZJ8eHalVoloIoaaATROH
spanish_df = spanish_df.set_index('id')
spanish_df = spanish_df.drop(columns=['books_labels.csv', 'codebook', 'language', 'book_file_name', 'file_name_translation', 'source', 'year', 'genre', 'genre-multilabel', 'testament', 'division'])
portuguese_df = portuguese_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'SF_2009-01-20_POR_ACF_(PORTUGUESE CORRIGIDA FIEL (1753_1995))']
portuguese_df = portuguese_df.set_index('id')
portuguese_df = portuguese_df.drop(columns=['books_labels.csv', 'codebook', 'language', 'book_file_name', 'file_name_translation', 'source', 'year', 'genre', 'genre-multilabel', 'testament', 'division'])
english_df = english_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'SF_2009-01-20_ENG_ASV_(AMERICAN STANDARD VERSION)'] # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.reddit.com/r/Reformed/comments/xoye82/most_accurate_spanish_bible_translations_spanish/&ved=2ahUKEwj7qbSKtbuFAxV4IEQIHbm6B_UQFnoECCwQAQ&usg=AOvVaw2YZJ8eHalVoloIoaaATROH
english_df = english_df.set_index('id')
english_df = english_df.drop(columns=['books_labels.csv', 'codebook', 'language', 'book_file_name', 'file_name_translation', 'source', 'year', 'genre', 'genre-multilabel', 'testament', 'division'])
german_df = german_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'SF_2009-01-20_GER_LUTH1912_(LUTHER 1912)']
german_df = german_df.set_index('id')
german_df = german_df.drop(columns=['books_labels.csv', 'codebook', 'language', 'book_file_name', 'file_name_translation', 'source', 'year', 'genre', 'genre-multilabel', 'testament', 'division'])


/var/folders/05/btfxjzv52_369qlngpc4v8fc0000gn/T/ipykernel_10877/1006097350.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  spanish_df = spanish_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'Nueva Biblia de las Américas (NBLA)'] # https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.reddit.com/r/Reformed/comments/xoye82/most_accurate_spanish_bible_translations_spanish/&ved=2ahUKEwj7qbSKtbuFAxV4IEQIHbm6B_UQFnoECCwQAQ&usg=AOvVaw2YZJ8eHalVoloIoaaATROH
/var/folders/05/btfxjzv52_369qlngpc4v8fc0000gn/T/ipykernel_10877/1006097350.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  portuguese_df = portuguese_df.sort_values(by=['file_name_translation', 'id'])[df['file_name_translation'] == 'SF_2009-01-20_POR_ACF_(PORTUGUESE CORRIGIDA FIEL (1753_1995))']
/var/folders/05/btfxjzv52_369qlngpc4v8fc0000gn/T/ipykernel_10877/1006097350.py:14: UserWarning: Boolean Series key

In [8]:
spanish_df.head(5)

,text
id,
b.1CH.001.001,"(A)Adán, Set, Enós,"
b.1CH.001.002,"Cainán, Mahalaleel, Jared,"
b.1CH.001.003,"Enoc, Matusalén, Lamec,"
b.1CH.001.004,"Noé, Sem, Cam y Jafet."
b.1CH.001.005,"(B)Los hijos de Jafet fueron Gomer, Magog, Mad..."


In [9]:
portuguese_df.head(5)

,text
id,
b.1CH.001.001,"ADÃO, Sete, Enos,"
b.1CH.001.002,"Cainã, Maalaleel, Jerede,"
b.1CH.001.003,"Enoque, Matusalém, Lameque,"
b.1CH.001.004,"Noé, Sem, Cão e Jafé."
b.1CH.001.005,"Os filhos de Jafé foram: Gomer, Magogue, Madai..."


In [10]:
english_df.head(5)

,text
id,
b.1CH.001.001,"Adam, Seth, Enosh,"
b.1CH.001.002,"Kenan, Mahalalel, Jared,"
b.1CH.001.003,"Enoch, Methuselah, Lamech,"
b.1CH.001.004,"Noah, Shem, Ham, and Japheth."
b.1CH.001.005,"The sons of Japheth: Gomer, and Magog, and Mad..."


In [11]:
german_df.head(5)

,text
id,
b.1CH.001.001,"Adam, Seth, Enos,"
b.1CH.001.002,"Kenan, Mahalaleel, Jared,"
b.1CH.001.003,"Henoch, Methusalah, Lamech,"
b.1CH.001.004,"Noah, Sem, Ham, Japheth."
b.1CH.001.005,"Die Kinder Japheths sind diese: Gomer, Magog, ..."


In [12]:
spanish_df = spanish_df.rename(columns={'text':'spanish_text'})
portuguese_df = portuguese_df.rename(columns={'text':'portuguese_text'})
english_df = english_df.rename(columns={'text':'english_text'})
german_df = german_df.rename(columns={'text':'german_text'})

multi_df = spanish_df.join(portuguese_df)
multi_df = multi_df.join(english_df)
multi_df = multi_df.join(german_df)
multi_df = multi_df.dropna()

print("Spanish:", multi_df['spanish_text']['b.GEN.001.001'], "\nPortuguese:", multi_df['portuguese_text']['b.GEN.001.001'], "\nEnglish:", multi_df['english_text']['b.GEN.001.001'], "\nGerman:", multi_df['german_text']['b.GEN.001.001'])

spanish_list = multi_df['spanish_text'].to_list()
portuguese_list = multi_df['portuguese_text'].to_list()
english_list = multi_df['english_text'].to_list()
german_list = multi_df['german_text'].to_list()

print('Spanish:', spanish_list[11], 'Portuguese:', portuguese_list[11])

Spanish: En el principio(A) Dios creó(B) los cielos y la tierra(C). 
Portuguese: NO princípio criou Deus os céus e a terra. 
English: In the beginning God created the heavens and the earth. 
German: Am Anfang schuf Gott Himmel und Erde.
Spanish: Canaán fue el padre de Sidón su primogénito, y de Het, Portuguese: E Canaã gerou a Sidom seu primogênito, e a Hete,


In [13]:
# Format and tokenize lists to make corpi
spanish_list = [regex.sub(r'\([a-zA-z0-9]\)', '', item) for item in spanish_list]
spanish_corpus = [nltk.tokenize.casual_tokenize(item) for item in spanish_list]
print(spanish_corpus[0])
portuguese_corpus = [nltk.tokenize.casual_tokenize(item) for item in portuguese_list]
print(portuguese_corpus[0])
english_corpus = [nltk.tokenize.casual_tokenize(item) for item in english_list]
print(english_corpus[0])
german_corpus = [nltk.tokenize.casual_tokenize(item) for item in german_list]
print(german_corpus[0])

['Adán', ',', 'Set', ',', 'Enós', ',']
['ADÃO', ',', 'Sete', ',', 'Enos', ',']
['Adam', ',', 'Seth', ',', 'Enosh', ',']
['Adam', ',', 'Seth', ',', 'Enos', ',']


In [14]:
# Select Portuguese texts
# https://stackoverflow.com/questions/17424182/extracting-all-rows-from-pandas-dataframe-that-have-certain-value-in-a-specific
# portuguese_df = df[df['language'] == 'POR']['text']
# portuguese_list = portuguese_df.to_list()

In [15]:
# Garbage collect old variables
del df
del spanish_list, portuguese_list, spanish_df, portuguese_df, english_list, german_list, english_df, german_df, multi_df
gc.collect()

0

In [16]:
# spanish_long_list = []
# for i in spanish_corpus:
#     spanish_long_list += i
# print(spanish_long_list[:200])
# spanish_long_list = [i.lower() for i in spanish_long_list]
# fd = FreqDist(spanish_long_list)
# spanish_vocabulary = set([text for text, i in fd.most_common(10000)])
# print(spanish_vocabulary)
# for i, text in enumerate(spanish_corpus):
#     for j, word in enumerate(text):
#         if word not in spanish_vocabulary:
#             spanish_corpus[i][j] = '[UNK]'

# portuguese_long_list = []
# for i in portuguese_corpus:
#     portuguese_long_list += i
# print(portuguese_long_list[:200])
# portuguese_long_list = [i.lower() for i in portuguese_long_list]
# fd = FreqDist(portuguese_long_list)
# portuguese_vocabulary = set([text for text, i in fd.most_common(10000)])
# print(portuguese_vocabulary)
# for i, text in enumerate(portuguese_corpus):
#     for j, word in enumerate(text):
#         if word not in portuguese_vocabulary:
#             portuguese_corpus[i][j] = '[UNK]'

# english_long_list = []
# for i in english_corpus:
#     english_long_list += i
# print(english_long_list[:200])
# english_long_list = [i.lower() for i in english_long_list]
# fd = FreqDist(english_long_list)
# english_vocabulary = set([text for text, i in fd.most_common(10000)])
# print(english_vocabulary)
# for i, text in enumerate(english_corpus):
#     for j, word in enumerate(text):
#         if word not in english_vocabulary:
#             english_corpus[i][j] = '[UNK]'
            
# german_long_list = []
# for i in german_corpus:
#     german_long_list += i
# print(german_long_list[:200])
# german_long_list = [i.lower() for i in german_long_list]
# fd = FreqDist(german_long_list)
# german_vocabulary = set([text for text, i in fd.most_common(10000)])
# print(german_vocabulary)
# for i, text in enumerate(german_corpus):
#     for j, word in enumerate(text):
#         if word not in german_vocabulary:
#             german_corpus[i][j] = '[UNK]'

In [17]:
english_corpus[12000]

['And',
 'it',
 'came',
 'to',
 'pass',
 'on',
 'the',
 'third',
 'day',
 ',',
 'when',
 'they',
 'were',
 'sore',
 ',',
 'that',
 'two',
 'of',
 'the',
 'sons',
 'of',
 'Jacob',
 ',',
 'Simeon',
 'and',
 'Levi',
 ',',
 'Dinah',
 '`',
 's',
 'brethren',
 ',',
 'took',
 'each',
 'man',
 'his',
 'sword',
 ',',
 'and',
 'came',
 'upon',
 'the',
 'city',
 'unawares',
 ',',
 'and',
 'slew',
 'all',
 'the',
 'males',
 '.']

In [18]:
# Prepare vocabulary
spanish_encoder_texts, spanish_input_texts, spanish_target_texts = [], [], []
portuguese_encoder_texts, portuguese_input_texts, portuguese_target_texts = [], [], []
english_encoder_texts, english_input_texts, english_target_texts = [], [], []
german_encoder_texts, german_input_texts, german_target_texts = [], [], []
spanish_vocabulary = set()
portuguese_vocabulary = set()
english_vocabulary = set()
german_vocabulary = set()
spanish_start_token = '[SPSTART]'
portuguese_start_token = '[POSTART]'
english_start_token = '[ENSTART]'
german_start_token = '[GESTART]'
stop_token = '[END]'
unknown_token = '[UNK]'
pad_token = '[PAD]'
spanish_vocabulary.add(spanish_start_token)
spanish_vocabulary.add(stop_token)
spanish_vocabulary.add(unknown_token)
spanish_vocabulary.add(pad_token)
portuguese_vocabulary.add(portuguese_start_token)
portuguese_vocabulary.add(stop_token)
portuguese_vocabulary.add(unknown_token)
portuguese_vocabulary.add(pad_token)
english_vocabulary.add(english_start_token)
english_vocabulary.add(stop_token)
english_vocabulary.add(unknown_token)
english_vocabulary.add(pad_token)
german_vocabulary.add(german_start_token)
german_vocabulary.add(stop_token)
german_vocabulary.add(unknown_token)
german_vocabulary.add(pad_token)

for spanish_text in spanish_corpus:
    spanish_encoder_texts.append([spanish_start_token] + spanish_text + [stop_token])
    spanish_input_texts.append([spanish_start_token] + spanish_text)
    spanish_target_texts.append(spanish_text + [stop_token])
    for char in spanish_text:
        if char not in spanish_vocabulary:
            spanish_vocabulary.add(char)

for portuguese_text in portuguese_corpus:
    portuguese_encoder_texts.append([portuguese_start_token] + portuguese_text + [stop_token])
    portuguese_input_texts.append([portuguese_start_token] + portuguese_text)
    portuguese_target_texts.append(portuguese_text + [stop_token])
    for char in portuguese_text:
        if char not in portuguese_vocabulary:
            portuguese_vocabulary.add(char)

for english_text in english_corpus:
    english_encoder_texts.append([english_start_token] + english_text + [stop_token])
    english_input_texts.append([english_start_token] + english_text)
    english_target_texts.append(english_text + [stop_token])
    for char in english_text:
        if char not in english_vocabulary:
            english_vocabulary.add(char)

for german_text in german_corpus:
    german_encoder_texts.append([german_start_token] + german_text + [stop_token])
    german_input_texts.append([german_start_token] + german_text)
    german_target_texts.append(german_text + [stop_token])
    for char in german_text:
        if char not in german_vocabulary:
            german_vocabulary.add(char)

unified_vocabulary = spanish_vocabulary.union(portuguese_vocabulary).union(english_vocabulary).union(german_vocabulary)

print(len(spanish_vocabulary), len(portuguese_vocabulary), len(english_vocabulary), len(german_vocabulary), len(unified_vocabulary))
print(spanish_encoder_texts[0], portuguese_input_texts[0], portuguese_target_texts[0])

26888 29155 13471 21854 83611
['[SPSTART]', 'Adán', ',', 'Set', ',', 'Enós', ',', '[END]'] ['[POSTART]', 'ADÃO', ',', 'Sete', ',', 'Enos', ','] ['ADÃO', ',', 'Sete', ',', 'Enos', ',', '[END]']


In [19]:
# Finish vocabulary
spanish_vocabulary = sorted(spanish_vocabulary)
portuguese_vocabulary = sorted(portuguese_vocabulary)
english_vocabulary = sorted(english_vocabulary)
german_vocabulary = sorted(german_vocabulary)

# Define maxima
spanish_vocab_size = len(spanish_vocabulary)
portuguese_vocab_size = len(portuguese_vocabulary)
spanish_vocab_size = len(spanish_vocabulary)
portuguese_vocab_size = len(portuguese_vocabulary)
unified_vocab_size = len(unified_vocabulary)
max_spanish_seq_length = max([len(txt) for txt in spanish_target_texts])
max_portuguese_seq_length = max([len(txt) for txt in portuguese_target_texts])
max_english_seq_length = max([len(txt) for txt in english_target_texts])
max_german_seq_length = max([len(txt) for txt in german_target_texts])
max_unified_seq_length = max(max_spanish_seq_length, max_portuguese_seq_length, max_english_seq_length, max_german_seq_length)

# Create indicies
spanish_token_index = dict([(token, i) for i, token in
                          enumerate(spanish_vocabulary)])
portuguese_token_index = dict([(token, i) for i, token in
                          enumerate(portuguese_vocabulary)])
unified_token_index = dict([(token, i) for i, token in
                          enumerate(unified_vocabulary)])
reverse_spanish_token_index = dict([(i, token) for token, i in
                          spanish_token_index.items()])
reverse_portuguese_token_index = dict([(i, token) for token, i in
                          portuguese_token_index.items()])
reverse_unified_token_index = dict([(i, token) for token, i in
                          unified_token_index.items()])
print(max_spanish_seq_length, max_unified_seq_length, max_portuguese_seq_length)

114 114 109


In [20]:
def convert_text_to_indices(texts, token_index, max_seq_length, pad_token='[PAD]'):
    data = np.zeros((len(texts), max_seq_length),
                    dtype='int32')
    for i, text in enumerate(texts):
        for t, token in enumerate(text):
            data[i, t] = token_index[token]
        for t in range(len(text), max_seq_length):
            data[i, t] = token_index[pad_token]
    
    return data

In [21]:
import numpy as np

spanish_portuguese_encoder_input_data = convert_text_to_indices(spanish_encoder_texts + portuguese_encoder_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length)+1)
spanish_portuguese_decoder_input_data = convert_text_to_indices(spanish_input_texts + portuguese_input_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length))
spanish_portuguese_decoder_target_data = convert_text_to_indices(spanish_target_texts + portuguese_target_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length))

portuguese_spanish_encoder_input_data = convert_text_to_indices(portuguese_encoder_texts + spanish_encoder_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length)+1)
portuguese_spanish_decoder_input_data = convert_text_to_indices(portuguese_input_texts + spanish_input_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length))
portuguese_spanish_decoder_target_data = convert_text_to_indices(portuguese_target_texts + spanish_target_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length))

final_encoder_input_data = convert_text_to_indices(spanish_encoder_texts + spanish_encoder_texts + portuguese_encoder_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length)+1)
final_decoder_input_data = convert_text_to_indices(english_input_texts + german_encoder_texts + portuguese_input_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length))
final_decoder_target_data = convert_text_to_indices(english_target_texts + german_encoder_texts + portuguese_target_texts, unified_token_index, max(max_spanish_seq_length, max_portuguese_seq_length))


# Convert sentences to numpy arrays
spanish_encoder_input_data = convert_text_to_indices(spanish_encoder_texts, unified_token_index, max_spanish_seq_length+1)
spanish_decoder_input_data = convert_text_to_indices(spanish_input_texts, unified_token_index, max_spanish_seq_length)
spanish_decoder_target_data = convert_text_to_indices(spanish_target_texts, unified_token_index, max_spanish_seq_length)

portuguese_encoder_input_data = convert_text_to_indices(portuguese_encoder_texts, unified_token_index, max_portuguese_seq_length+1)
portuguese_decoder_input_data = convert_text_to_indices(portuguese_input_texts, unified_token_index, max_portuguese_seq_length)
portuguese_decoder_target_data = convert_text_to_indices(portuguese_target_texts, unified_token_index, max_portuguese_seq_length)

english_encoder_input_data = convert_text_to_indices(english_encoder_texts, unified_token_index, max_english_seq_length+1)
english_decoder_input_data = convert_text_to_indices(english_input_texts, unified_token_index, max_english_seq_length)
english_decoder_target_data = convert_text_to_indices(english_target_texts, unified_token_index, max_english_seq_length)

german_encoder_input_data = convert_text_to_indices(german_encoder_texts, unified_token_index, max_german_seq_length+1)
german_decoder_input_data = convert_text_to_indices(german_input_texts, unified_token_index, max_german_seq_length)
german_decoder_target_data = convert_text_to_indices(german_target_texts, unified_token_index, max_german_seq_length)

# spanish_encoder_input_data = np.zeros((len(spanish_encoder_texts), max_spanish_seq_length),
#                                dtype='int32')
# spanish_decoder_input_data = np.zeros((len(spanish_input_texts), max_spanish_seq_length),
#                                dtype='int32')
# spanish_decoder_target_data = np.zeros((len(spanish_target_texts), max_spanish_seq_length),
#                                dtype='int32')

# portuguese_encoder_input_data = np.zeros((len(portuguese_encoder_texts), max_portuguese_seq_length),
#                                dtype='int32')
# portuguese_decoder_input_data = np.zeros((len(portuguese_input_texts), max_portuguese_seq_length),
#                                dtype='int32')
# portuguese_decoder_target_data = np.zeros((len(portuguese_target_texts), max_portuguese_seq_length),
#                                dtype='int32')



# for i, (input_text, target_text) in enumerate(zip(spanish_input_texts, spanish_target_texts)):
#     for t, token in enumerate(input_text):
#         spanish_encoder_input_data[
#             i, t] = unified_token_index[token]
#     for t, token in enumerate(target_text):
#         spanish_decoder_input_data[
#             i, t] = unified_token_index[token]
#         if t > 0:
#             spanish_decoder_target_data[i, t - 1] = spanish_token_index[token]

# for i, (input_text, target_text) in enumerate(zip(portuguese_input_texts, portuguese_target_texts)):
#     for t, token in enumerate(input_text):
#         portuguese_encoder_input_data[
#             i, t] = unified_token_index[token]
#     for t, token in enumerate(target_text):
#         portuguese_decoder_input_data[
#             i, t] = unified_token_index[token]
#         if t > 0:
#             portuguese_decoder_target_data[i, t - 1] = portuguese_token_index[token]

print([reverse_unified_token_index[value] for value in portuguese_encoder_input_data[0]])
print([reverse_unified_token_index[value] for value in portuguese_decoder_input_data[0]])
print([reverse_unified_token_index[value] for value in portuguese_decoder_target_data[0]])
print([reverse_unified_token_index[value] for value in spanish_encoder_input_data[0]])
print([reverse_unified_token_index[value] for value in spanish_decoder_input_data[0]])
print([reverse_unified_token_index[value] for value in spanish_decoder_target_data[0]])
print([reverse_unified_token_index[value] for value in german_encoder_input_data[0]])
print([reverse_unified_token_index[value] for value in german_decoder_input_data[0]])
print([reverse_unified_token_index[value] for value in german_decoder_target_data[0]])
print([reverse_unified_token_index[value] for value in english_encoder_input_data[0]])
print([reverse_unified_token_index[value] for value in english_decoder_input_data[0]])
print([reverse_unified_token_index[value] for value in english_decoder_target_data[0]])

['[POSTART]', 'ADÃO', ',', 'Sete', ',', 'Enos', ',', '[END]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[POSTART]', 'ADÃO'

In [22]:
# Display some input
spanish_encoder_input_data

array([[52786, 11589, 20334, ..., 66366, 66366, 66366],
       [52786, 13923, 20334, ..., 66366, 66366, 66366],
       [52786, 67061, 20334, ..., 66366, 66366, 66366],
       ...,
       [52786, 40345, 53378, ..., 66366, 66366, 66366],
       [52786, 65023, 40120, ..., 66366, 66366, 66366],
       [52786, 78647,   501, ..., 66366, 66366, 66366]], dtype=int32)

In [23]:
# Define hyperparameters
batch_size = 32
epochs = 10
num_neurons = 256

In [24]:
num_layers = 3
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [25]:
# Import from customized file
from transformers import *

In [26]:
encoder = Encoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)

spanish_decoder = Decoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)

english_decoder = Decoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)

portuguese_decoder = Decoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)

german_decoder = Decoder(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=unified_vocab_size,
    dropout_rate=dropout_rate)

2024-04-25 03:18:33.855094: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-04-25 03:18:33.855115: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-04-25 03:18:33.855119: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-04-25 03:18:33.855136: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-25 03:18:33.855148: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [27]:
# Create the transformers from the encoder and decoder
# transformer = Transformer(num_layers=num_layers,
#     d_model=d_model,
#     num_heads=num_heads,
#     dff=dff,
#     input_vocab_size=unified_vocab_size,
#     target_vocab_size=unified_vocab_size,
#     dropout_rate=dropout_rate)
spanish_transformer = ComposedTransformer(encoder, spanish_decoder, unified_vocab_size)
english_transformer = ComposedTransformer(encoder, english_decoder, unified_vocab_size)
portuguese_transformer = ComposedTransformer(encoder, portuguese_decoder, unified_vocab_size)
german_transformer = ComposedTransformer(encoder, german_decoder, unified_vocab_size)

In [28]:
# Define learning rate
learning_rate = CustomSchedule(d_model)

spanish_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
portuguese_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
english_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
german_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [29]:
# Compile both models
spanish_transformer.compile(
    loss=masked_loss,
    optimizer=spanish_optimizer,
    metrics=[masked_accuracy])
english_transformer.compile(
    loss=masked_loss,
    optimizer=english_optimizer,
    metrics=[masked_accuracy])
portuguese_transformer.compile(
    loss=masked_loss,
    optimizer=portuguese_optimizer,
    metrics=[masked_accuracy])
german_transformer.compile(
    loss=masked_loss,
    optimizer=german_optimizer,
    metrics=[masked_accuracy])

In [30]:
# Prepare training data
# dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[1000:2000], portuguese_decoder_input_data[1000:2000]), portuguese_decoder_target_data[1000:2000]))
# batched_dataset = dataset.batch(batch_size)
# spanish_dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, spanish_decoder_input_data), spanish_decoder_target_data))
# portuguese_dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data, spanish_decoder_input_data), spanish_decoder_target_data))
# spanish_batched_dataset = spanish_dataset.batch(batch_size)
# portuguese_batched_dataset = portuguese_dataset.batch(batch_size)

In [31]:
# Give a summary of the transformer
english_transformer.summary()

Model: "composed_transformer_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_1 (Decoder)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Define a translator
class Translator(tf.Module):
  def __init__(self, transformer, token_index, reverse_token_index, input_start_char, output_start_char):
    self.transformer = transformer
    self.token_index = token_index
    self.reverse_token_index = reverse_token_index
    self.input_start_char = input_start_char
    self.output_start_char = output_start_char

  def __call__(self, sentence, max_length):
    # The input sentence is Portuguese, hence adding the `[START]` and `[END]` tokens.
    # assert isinstance(sentence, tf.Tensor)
    # if len(sentence.shape) == 0:
    #   sentence = sentence[tf.newaxis]
    tokenized_sentence = [self.input_start_char] + nltk.tokenize.casual_tokenize(sentence.lower()) + ['[END]']
    sentence_tensor = np.zeros(len(tokenized_sentence), dtype='int64')
    for t, token in enumerate(tokenized_sentence):
      if token in self.token_index:
        sentence_tensor[t] = self.token_index[token]
      else:
        sentence_tensor[t] = self.token_index['[UNK]']
    #sentence = self.tokenizers.pt.tokenize(sentence).to_tensor()

    encoder_input = sentence_tensor[tf.newaxis]
    
    # As the output language is English, initialize the output with the
    # English `[START]` token.
    #start_end = self.tokenizers.en.tokenize([''])[0]
    start = self.token_index[self.output_start_char]
    end = self.token_index['[END]']

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)
    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())[tf.newaxis]
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0][0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`. https://www.tensorflow.org/api_docs/python/tf/cast
    text = [self.reverse_token_index[item.numpy()] for item in output]  # Shape: `()`.

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    # self.transformer([encoder_input, output[:,:-1]], training=False)
    # attention_weights = self.transformer.decoder.last_attn_scores

    return text

In [33]:
# Train the transformers
tf.config.run_functions_eagerly(True)
# transformer.fit(batched_dataset, epochs=epochs)

In [34]:
print(len(spanish_encoder_input_data), len(portuguese_decoder_input_data), len(portuguese_decoder_target_data))

28391 28391 28391


In [35]:
from nltk.translate import meteor_score
import random
def calculate_average_meteor(input_dataset, confirmation_dataset, translator, numtests, maxlen):
    score = 0
    i = 0
    while i < numtests:
        randindex = random.randrange(0, len(input_dataset))
        if len(input_dataset[randindex]) <= maxlen:
            expected = confirmation_dataset[randindex]
            result = translator(' '.join(input_dataset[randindex]), len(confirmation_dataset[randindex]))
            score += meteor_score.single_meteor_score(result[1:], expected)
            i += 1
    return score / i

In [36]:
# sentence = 'En el principio Dios creó los cielos y la tierra.'
# translator = Translator(transformer, unified_token_index, reverse_unified_token_index, '[SPSTART]', '[POSTART]')
# # portuguese_translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
# portuguese_translated_text = translator(
#     sentence,20)
# for i in portuguese_translated_text:
#     try:
#         print(reverse_unified_token_index[i], end=' ')
#     except Exception:
#         print(i)
#         continue
# print(calculate_average_meteor(spanish_corpus, portuguese_corpus, translator, 10, 20))

In [74]:
for t in range(6):
    print(f"Epoch {t}:")
    for i in range(56):
        dataset = tf.data.Dataset.from_tensor_slices(((portuguese_encoder_input_data[i*500:i*500+500], portuguese_decoder_input_data[i*500:i*500+500]), portuguese_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        portuguese_transformer.fit(batched_dataset)

        dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[i*500:i*500+500], spanish_decoder_input_data[i*500:i*500+500]), spanish_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        english_transformer.fit(batched_dataset)

        dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[i*500:i*500+500], english_decoder_input_data[i*500:i*500+500]), english_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        german_transformer.fit(batched_dataset)

        dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[i*500:i*500+500], german_decoder_input_data[i*500:i*500+500]), german_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        spanish_transformer.fit(batched_dataset)
        
        dataset = tf.data.Dataset.from_tensor_slices(((portuguese_encoder_input_data[i*500:i*500+500], portuguese_decoder_input_data[i*500:i*500+500]), portuguese_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        portuguese_transformer.fit(batched_dataset)
        sentence = 'En el principio Dios creó los cielos y la tierra.'
        translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, '[SPSTART]', '[POSTART]')
        #print(calculate_average_meteor(spanish_corpus, portuguese_corpus, translator, 100, 20))
        print(translator(sentence,20))

        # dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[i*2000:i*2000+2000], german_decoder_input_data[i*2000:i*2000+2000]), german_decoder_target_data[i*2000:i*2000+2000]))
        # batched_dataset = dataset.batch(batch_size)
        # transformer.fit(batched_dataset)
        # dataset = tf.data.Dataset.from_tensor_slices(((english_encoder_input_data[i*2000:i*2000+2000], spanish_decoder_input_data[i*2000:i*2000+2000]), spanish_decoder_target_data[i*2000:i*2000+2000]))
        # batched_dataset = dataset.batch(batch_size)
        # transformer.fit(batched_dataset)
        # dataset = tf.data.Dataset.from_tensor_slices(((portuguese_encoder_input_data[i*2000:i*2000+2000], portuguese_decoder_input_data[i*2000:i*2000+2000]), portuguese_decoder_target_data[i*2000:i*2000+2000]))
        # batched_dataset = dataset.batch(batch_size)
        # transformer.fit(batched_dataset)
    sentence = 'En el principio Dios creó los cielos y la tierra.'
    translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, '[SPSTART]', '[POSTART]')
    #print(calculate_average_meteor(spanish_corpus, portuguese_corpus, translator, 100, 20))
    print(translator(sentence,20))
    # portuguese_translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
    # portuguese_translated_text = translator(
    #     sentence,20)
    # for i in portuguese_translated_text:
    #     try:
    #         print(reverse_unified_token_index[i], end=' ')
    #     except Exception:
    #         print(i)
    #         continue
        #transformer.save_weights(f'./models/spanish-portuguese_full_epoch{t}.weights.h5')

Epoch 0:


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer doe

16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 11.3090 - masked_accuracy: 0.0000e+00


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'causal_self_attention_15' (of type CausalSelfAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'decoder_layer_15' (of type DecoderLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 11.3220 - masked_accuracy: 0.0000e+00


...

/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'causal_self_attention_12' (of type CausalSelfAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'decoder_layer_12' (of type DecoderLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 11.2619 - masked_accuracy: 0.0000e+00
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 11.2204 - masked_accuracy: 0.0591


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


['[POSTART]', 'Stimme', 'Stimme', 'erwerben', '[PAD]', 'canções', 'canções', 'canções', 'canções', 'hallaste', 'Esceva', 'envejecer', 'Stimme', 'envejecer', 'Raubebald', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 11.0627 - masked_accuracy: 0.7440
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 11.2447 - masked_accuracy: 0.0219
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - loss: 11.2234 - masked_accuracy: 0.0833
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: 11.1898 - masked_accuracy: 0.1404
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 10.9030 - masked_accuracy: 0.7450
['[POSTART]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 10.7713 - masked_accuracy: 0.7570
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 11.0923 - masked_accuracy: 0.6782
16/16 ━━━━━━━━━

KeyboardInterrupt: 

In [37]:
print('Pre-training:')
dataset = tf.data.Dataset.from_tensor_slices(((portuguese_encoder_input_data[0:1000], portuguese_decoder_input_data[0:1000]), portuguese_decoder_target_data[0:1000]))
dataset = dataset.shuffle(dataset.cardinality())
batched_dataset = dataset.batch(batch_size)
portuguese_transformer.fit(batched_dataset)
for t in range(6):
    print(f"Epoch {t}:")
    for i in range(56):
        dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[i*500:i*500+500], spanish_decoder_input_data[i*500:i*500+500]), spanish_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        english_transformer.fit(batched_dataset)

        dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[i*500:i*500+500], english_decoder_input_data[i*500:i*500+500]), english_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        german_transformer.fit(batched_dataset)

        dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[i*500:i*500+500], german_decoder_input_data[i*500:i*500+500]), german_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        spanish_transformer.fit(batched_dataset)
        
        dataset = tf.data.Dataset.from_tensor_slices(((portuguese_encoder_input_data[i*500:i*500+500], portuguese_decoder_input_data[i*500:i*500+500]), portuguese_decoder_target_data[i*500:i*500+500]))
        dataset = dataset.shuffle(dataset.cardinality())
        batched_dataset = dataset.batch(batch_size)
        portuguese_transformer.fit(batched_dataset)
        sentence = 'En el principio Dios creó los cielos y la tierra.'
        translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, '[SPSTART]', '[POSTART]')
        #print(calculate_average_meteor(spanish_corpus, portuguese_corpus, translator, 100, 20))
        print(translator(sentence,20))

        # dataset = tf.data.Dataset.from_tensor_slices(((spanish_encoder_input_data[i*2000:i*2000+2000], german_decoder_input_data[i*2000:i*2000+2000]), german_decoder_target_data[i*2000:i*2000+2000]))
        # batched_dataset = dataset.batch(batch_size)
        # transformer.fit(batched_dataset)
        # dataset = tf.data.Dataset.from_tensor_slices(((english_encoder_input_data[i*2000:i*2000+2000], spanish_decoder_input_data[i*2000:i*2000+2000]), spanish_decoder_target_data[i*2000:i*2000+2000]))
        # batched_dataset = dataset.batch(batch_size)
        # transformer.fit(batched_dataset)
        # dataset = tf.data.Dataset.from_tensor_slices(((portuguese_encoder_input_data[i*2000:i*2000+2000], portuguese_decoder_input_data[i*2000:i*2000+2000]), portuguese_decoder_target_data[i*2000:i*2000+2000]))
        # batched_dataset = dataset.batch(batch_size)
        # transformer.fit(batched_dataset)
    sentence = 'En el principio Dios creó los cielos y la tierra.'
    translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, '[SPSTART]', '[POSTART]')
    print(calculate_average_meteor(spanish_corpus, portuguese_corpus, translator, 100, 20))
    print(translator(sentence,20))
    # portuguese_translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
    # portuguese_translated_text = translator(
    #     sentence,20)
    # for i in portuguese_translated_text:
    #     try:
    #         print(reverse_unified_token_index[i], end=' ')
    #     except Exception:
    #         print(i)
    #         continue
        #transformer.save_weights(f'./models/spanish-portuguese_full_epoch{t}.weights.h5')

Pre-training:


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer doe

32/32 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - loss: 11.2687 - masked_accuracy: 0.0239
Epoch 0:


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'causal_self_attention_3' (of type CausalSelfAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'decoder_layer_3' (of type DecoderLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 11.3570 - masked_accuracy: 0.0000e+00


...

/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'causal_self_attention' (of type CausalSelfAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/layers/layer.py:856: UserWarning: Layer 'decoder_layer' (of type DecoderLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 11.3222 - masked_accuracy: 0.0000e+00
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 11.0392 - masked_accuracy: 0.7881


/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


['[POSTART]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 11.2698 - masked_accuracy: 0.0095
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - loss: 11.2189 - masked_accuracy: 0.0681
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: 11.2413 - masked_accuracy: 0.0334
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss: 10.9117 - masked_accuracy: 0.7383
['[POSTART]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - loss: 11.1014 - masked_accuracy: 0.6893
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - loss: 11.0642 - masked_accuracy: 0.7139
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: 11.0837 - masked_accuracy: 0.7006
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - loss

KeyboardInterrupt: 

In [38]:
sentence = 'En el principio Dios creó los cielos y la tierra.'
translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, '[SPSTART]', '[POSTART]')
print(calculate_average_meteor(spanish_corpus, portuguese_corpus, translator, 100, 20))
print(translator(sentence,20))

0.13431899827146215
['[POSTART]', 'em', 'seus', 'Santo', 'escolheu', 'escolheu', 'SENHOR', 'teu', 'estatutos', 'e', 'terra', '.', '[END]']


In [ ]:
tf.saved_model.save(transformer, export_dir=f'./translators/{t}')
transformer = tf.saved_model.load(export_dir='./translators/0')

INFO:tensorflow:Assets written to: ./translators/1/assets


INFO:tensorflow:Assets written to: ./translators/1/assets


In [ ]:
# spanish_transformer.save_weights('./models/spanish120epoch.weights.h5')
# portuguese_transformer.save_weights('./models/portuguese120epoch.weights.h5')
#transformer.save_weights('./models/spanish-portuguese_full_epoch59.weights.h5')
# transformer.save('./models/spanish-portuguese_full_test.keras') # https://discuss.tensorflow.org/t/save-a-tensorflow-model-with-a-transformer-layer/7221
# transformer = tf.keras.models.load_model('./models/spanish-portuguese_full_test.keras') # https://discuss.tensorflow.org/t/save-a-tensorflow-model-with-a-transformer-layer/7221
transformer.load_weights('./models/spanish-portuguese_full_epoch9.weights.h5')

/Users/luke/Documents/Code/CS362N Natural Language Processing/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 1 variables whereas the saved optimizer has 345 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Define a translator
class Translator(tf.Module):
  def __init__(self, transformer, input_token_index, reverse_input_token_index, output_token_index, reverse_output_token_index):
    self.transformer = transformer
    self.input_token_index = input_token_index
    self.output_token_index = output_token_index
    self.reverse_input_token_index = reverse_input_token_index
    self.reverse_output_token_index = reverse_output_token_index

  def __call__(self, sentence, max_length):
    # The input sentence is Portuguese, hence adding the `[START]` and `[END]` tokens.
    # assert isinstance(sentence, tf.Tensor)
    # if len(sentence.shape) == 0:
    #   sentence = sentence[tf.newaxis]
    tokenized_sentence = ['[SPSTART]'] + nltk.tokenize.casual_tokenize(sentence) + ['[END]']
    sentence_tensor = np.zeros(max_length, dtype='int64')
    for t, token in enumerate(tokenized_sentence):
      sentence_tensor[t] = self.input_token_index[token]
    #sentence = self.tokenizers.pt.tokenize(sentence).to_tensor()

    encoder_input = sentence_tensor[tf.newaxis]
    
    # As the output language is English, initialize the output with the
    # English `[START]` token.
    #start_end = self.tokenizers.en.tokenize([''])[0]
    start = self.input_token_index['[POSTART]']
    end = self.output_token_index['[END]']

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)
    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())[tf.newaxis]
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0][0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`. https://www.tensorflow.org/api_docs/python/tf/cast
    text = [item.numpy() for item in output]  # Shape: `()`.

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    # self.transformer([encoder_input, output[:,:-1]], training=False)
    # attention_weights = self.transformer.decoder.last_attn_scores

    return text

In [ ]:
# Test the translator
sentence = 'Estas son las palabras de Amós, que era un pastor de Tecoa'
translator = Translator(transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
# portuguese_translator = Translator(portuguese_transformer, unified_token_index, reverse_unified_token_index, unified_token_index, reverse_unified_token_index)
portuguese_translated_text = translator(
    sentence,15)
# portuguese_translated_text = portuguese_translator(
#     sentence,15)

TypeError: '_UserObject' object is not callable

In [ ]:
# Print the raw lists
#print(spanish_translated_text)
print(portuguese_translated_text)

[43282, 26068, 21446, 8956, 2923, 1776, 44249, 33554, 44249, 14713, 13664, 19175, 40547, 48550, 16274, 19832]


In [ ]:
# Print the sentences themselves
# for i in range(20):
#     try:
#         print(reverse_spanish_token_index[spanish_translated_text[i]], end=' ')
#     except Exception:
#         continue
# print()
for i in portuguese_translated_text:
    try:
        print(reverse_unified_token_index[i], end=' ')
    except Exception:
        print(i)
        continue

[POSTART] progresarán hendidura envidies composto ventre exacta intentaram exacta hendiste revés sereditas zelo secou Maldade - 

In [ ]:
del translator
gc.collect()

2867